# Create views

Create new [view items](https://doc.arcgis.com/en/arcgis-online/manage-data/create-hosted-views.htm) from feature layer items. View items are akin to database views--they can have separate permissions and capabilities as well as limited access to a subset of the underlying data. This means you can maintain a single, authoritative layer while controlling how different groups of users can access and interact with that layer.

In [1]:
# common imports
import os
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection, FeatureLayer

***Note**, if you are unable to import local `partnerutils`, **copy the following functions** from [`feature_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/feature_utils.py)

In [2]:
from partnerutils.feature_utils import sdf_from_xyz

**GIS**: Your GIS instance, parameter information [here](https://developers.arcgis.com/python/guide/using-the-gis/). Not specifying a password creates a password prompt

In [3]:
gis = GIS(username="mpayson_startups")

Enter password:  ········


### Create demo layer

In [4]:
df = pd.read_csv('../sample_data/NYC_Restaurant_Inspections.csv', encoding='utf-8')
sdf = sdf_from_xyz(df, 'Longitude', 'Latitude')
lyr_item = sdf.spatial.to_featurelayer(
    'Base demo layer',
    tags="esri_partner_tools_sample"
)
lyr_item

<Item title:"Base demo layer" type:Feature Layer Collection owner:mpayson_startups>

## Execution

### Create view item - [doc](https://developers.arcgis.com/python/api-reference/arcgis.features.managers.html?highlight=create_view#arcgis.features.managers.FeatureLayerCollectionManager.create_view)

In [5]:
# A feature service can have many layers, so a FeatureLayerCollection represents
# the underlying service. Often, like through the Python API, newly created layers
# create new services as well. So the desired layer is the first layer in the service.
flc = FeatureLayerCollection.fromitem(lyr_item)
view_item = flc.manager.create_view('Base demo view')
view_item.update(item_properties={'tags': 'esri_partner_tools_sample'})
view_item

<Item title:"Base demo view" type:Feature Layer Collection owner:mpayson_startups>

In [6]:
view_lyr = view_item.layers[0]

# create a reference sdf so we can compare what data is available in the view
def print_reference(ref_view_lyr):
    ref_sdf = ref_view_lyr.query().sdf
    print("# features: {}, # fields: {}".format(len(ref_sdf), len(ref_sdf.columns)))

print_reference(view_lyr)

# features: 5000, # fields: 27


### Update visible data in view

#### Fields
Update visible fields, only specify visible fields. Note, objectIdField must be visible

In [7]:
keep_field_names = {'GRADE', 'VIOLATION_', 'SCORE'}
keep_field_names.add(view_lyr.properties.objectIdField)
fields = list(map(
    lambda f: {
        'name': f.name,
        'visible': True if f.name in keep_field_names else False
    },
    view_lyr.properties.fields
))
view_lyr.manager.update_definition({"fields": fields})

print_reference(view_lyr)

# features: 5000, # fields: 5


#### Features by attribute
Update visible rows based on attribute [SQL where clause](https://pro.arcgis.com/en/pro-app/help/mapping/navigation/sql-reference-for-elements-used-in-query-expressions.htm)

In [8]:
where = "GRADE = 'A'"
view_lyr.manager.update_definition({"viewDefinitionQuery": where})

print_reference(view_lyr)

# features: 3955, # fields: 5


#### Features by geometry
Update visible rows based on a geography. A couple notes, can pass in a JSON representation of an [envelope](https://developers.arcgis.com/python/api-reference/arcgis.geometry.html#envelope) or [polygon](https://developers.arcgis.com/python/api-reference/arcgis.geometry.html#polygon). There's probably overhead to arbitray or complex polygons compared to envelopes. Also, the geometry coordinates should be specified in the same coordinate reference system as the map, this is often WebMercator (`wkid #102100`).

In [9]:
# Get Manhattan geometry, this is on the complex side. Data pulled from Hub
# https://hub.arcgis.com/datasets/498c7ff03780407494301c23cb59b899_0
buro_lyr = FeatureLayer('https://services1.arcgis.com/oOUgp466Coyjcu6V/arcgis/rest/services/NYC_Boroughs/FeatureServer/0')
buro_fs = buro_lyr.query(out_sr=102100)
manhattan_feature = list(filter(
    lambda f: f.attributes['boro_name'] == 'Manhattan',
    buro_fs.features
))[0]
manhattan_geom = manhattan_feature.geometry
manhattan_geom['spatialReference'] = buro_fs.spatial_reference

geo_definition = {
    "filter": {
        "operator": "esriSpatialRelIntersects",
        "value": {
            "geometryType": buro_fs.geometry_type,
            "geometry": manhattan_geom
        }
    }
}
view_lyr.manager.update_definition({"viewLayerDefinition": geo_definition})

print_reference(view_lyr)

# features: 1532, # fields: 5


## Clean up

In [10]:
delete_items = gis.content.search("tags:esri_partner_tools_sample")
gis.content.delete_items(delete_items)

True